In [1]:
# Combine two dataframes into one common dataframe,
# and dump to SQL file for upload to Web app
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For munging
import re
import json

In [2]:
yalePath = os.path.join(PROJ_ROOT, "data", "interim", "yale_bios_processed.csv")
harvardPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bios_processed.csv")

yale = pd.read_csv(yalePath)
harvard = pd.read_csv(harvardPath)

In [3]:
yale.shape

(3004, 18)

In [4]:
harvard.shape

(4270, 19)

In [5]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio'],
      dtype='object')

In [6]:
harvard.columns

Index([u'Name', u'B/T', u'City', u'Event', u'High School', u'Hometown', u'Ht.',
       u'Name_link', u'No.', u'Position', u'Region', u'S/C', u'Weapon', u'Wt.',
       u'Yr.', u'season', u'sport', u'headers', u'Bio'],
      dtype='object')

In [9]:
yale.head(20)

,Name,B/T,City,Cl.,Events,High School,Hometown/High School,Hometown/Region,Ht.,Name_link,No.,Pos.,Region,Weapon,Wt.,season,sport,Bio
0,A.J. Haase,NaN,Bonne Terre,Fr.,NaN,North County,"Bonne Terre, MO / North County","Bonne Terre, MO",6-4,/sports/m-footbl/2006-07/bios/haase_a.j.00.html,NaN,TE,MO,NaN,245.0,2006-10,Football,2010-11: NCAA East Regional All-TournamentTeam...
1,AJ Edwards,NaN,Seattle,Fr.,NaN,South Kent School,"Seattle, Wash. / South Kent School","Seattle, Wash.",6-5,/sports/m-baskbl/2013-14/bios/edwards_aj_geat,25.0,G,Wash.,NaN,190.0,2013-17,Basketball,2010: Appeared in five games... Recorded a 9.0...
2,Aarica West,NaN,Buena Park,Freshman,NaN,Vista Murrieta,"Buena Park, Calif. / Vista Murrieta","Buena Park, Calif.",5-10,/sports/w-baskbl/2009-10/bios/west_aarica,31.0,G/F,Calif.,NaN,NaN,2009-12,Basketball,2014-15 (Senior): Named captain of Yale Heavyw...
3,Aaron Ault,NaN,Altamonte Springs,Fr.,NaN,Lake Brantley,"Altamonte Springs, FL / Lake Brantley","Altamonte Springs, FL",6-3,/sports/m-footbl/2012-13/bios/ault_aaron_lcur,93.0,LB,FL,NaN,194.0,2012-16,Football,Notes: Owns school record for career goals aga...
4,Aaron Bosgang,NaN,Port Washington,Fr.,NaN,Paul D. Schreiber High School,"Port Washington, N.Y. / Paul D. Schreiber High...","Port Washington, N.Y.",NaN,/sports/m-crewlt/2016-17/bios/bosgang_aaron_uh2u,NaN,NaN,N.Y.,NaN,NaN,2016-17,Crew (Lightweight),2015: Played in all 10 games…Had 11 solo tackl...
5,Aaron Fuchs,NaN,Somerset West,Senior,NaN,Panel Vallei,"Somerset West, South Africa / Panel Vallei","Somerset West, South Africa",NaN,/sports/m-squash/2009-10/bios/fuchs_aaron,NaN,NaN,South Africa,NaN,NaN,2009-10,Squash,2014-15: Won a bronze medal with the second va...
6,Aaron Greenberg,NaN,Eden Prairie,Fr.,Freestyle/Breaststroke/Backstroke,Eden Prairie High School,"Eden Prairie, Minn. / Eden Prairie High School","Eden Prairie, Minn.",NaN,/sports/m-swim/2013-14/bios/greenberg_aaron_ifc2,NaN,NaN,Minn.,NaN,NaN,2013-17,Swimming & Diving,2013-14: Saw action in the third varsity eight...
7,Aaron Jones,NaN,"St. Catharines, Ont., Canada",Fr.,NaN,Governor Simcoe,"St. Catharines, Ont., Canada / Governor Simcoe","St. Catharines, Ont., Canada",NaN,/sports/m-crewlt/2013-14/bios/jones_aaron_g4f2,NaN,NaN,NOREGION?,NaN,NaN,2013-17,Crew (Lightweight),2011: Starting holder for placekicks... Had on...
8,Aaron Rodriguez,NaN,Cypress,Jr.,NaN,Cy-Fair,"Cypress, TX / Cy-Fair","Cypress, TX",5-5,/sports/m-soccer/2003-04/bios/rodriguez_aaron0...,7.0,Midfield,TX,NaN,135.0,2003-05,Soccer,Yale PRs800m 1:58.251000m 2:33.321500m 4:03.43...
9,Aaron Seriff-Cullick,NaN,-,Fr.,Diving,-,- / -,-,NaN,/sports/m-swim/2009-10/bios/seriff-cullick_aar...,NaN,NaN,-,NaN,NaN,2009-13,Swimming & Diving,2011-12: Sat in the third varsity eight.Porter...


In [8]:
harvard.head()

,Name,B/T,City,Event,High School,Hometown,Ht.,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,headers,Bio
0,A.J. Jaffe,NaN,St. Charles,NaN,Marmion Academy,"St. Charles, Ill.",NaN,/sports/wrest/2016-17/bios/jaffe_aj,NaN,141,Ill.,NaN,NaN,NaN,Freshman,2016-17,wrest,"{u'Hometown:': u'St. Charles, Ill.', u'Positio...",Before Harvard Four-time letterwinner at Marmi...
1,A.J. Kennedy,NaN,Fonthill,NaN,Notre Dame College,"Fonthill, Ont.",6-3,/sports/fball/2003-04/bios/a.j._kennedy,98,DE,Ont.,NaN,NaN,220,Freshman,2003-04,fball,"{u'Position:': u'DE', u'Year:': u'Freshman', u...",NaN
2,AJ Carvalho,NaN,Swansea,NaN,Joseph Case,"Swansea, Mass.",NaN,/sports/mcrew-lw/2010-11/bios/varsity/Carvalho...,NaN,NaN,Mass.,NaN,NaN,NaN,Sophomore,2010-13,mcrew-lw,"{u'name': u'Antonio Carvalho', u'Year:': u'Sop...",Sophomore • 2011Did not see spring race action...
3,Aaki Vora,NaN,Mumbai,Freestyle,The Cathedral and John Connon School,"Mumbai, India",NaN,/sports/wswimdive/2016-17/bios/Vora_Aaki,NaN,NaN,India,NaN,NaN,NaN,Freshman,2016-17,wswimdive,"{u'Hometown:': u'Mumbai, India', u'Position:':...",Before Harvard Best female athlete at the 2015...
4,Aaron Byrd,NaN,Abilene,NaN,Cooper,"Abilene, Texas",6-2,/sports/fball/2001-02/bios/aaron_byrd,3,DB,Texas,NaN,NaN,190,Freshman,2001-03,fball,"{u'Position:': u'DB', u'Year:': u'Freshman', u...",NaN


Definitive list of columns:
1. Name
2. B/T
3. City
4. Events
5. High School
6. Hometown
7. Height
8. No.
9. Position
10. Region
11. S/C
12. Weapon
13. Weight
14. Year
15. Active Seasons
16. Sport
17. Headers
18. Bio

Next step involves adding blank data to tables when either does not contain the above columns, and cleaning up some of the data to ensure consistency.

In [78]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio'],
      dtype='object')

In [79]:
# Insert missing columns
yale["S/C"] = None
yale["Headers"] = None
yale["College"] = "Yale"
# Re-order, then rename
# Yale
yaleColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown/High School",
               "Ht.", "No.", "Pos.", "Region", "S/C", "Weapon", "Wt.", "Cl.", "season", "sport", "Headers", "Bio", "College"]

In [80]:
yale = yale[yaleColumns]

In [81]:
harvard.columns

Index([u'Name', u'B/T', u'City', u'Event', u'High School', u'Hometown', u'Ht.',
       u'Name_link', u'No.', u'Position', u'Region', u'S/C', u'Weapon', u'Wt.',
       u'Yr.', u'season', u'sport', u'headers', u'Bio'],
      dtype='object')

In [82]:
harvard["College"] = "Harvard"
harvardColumns = ["Name", "B/T", "City", "Event", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", "Yr.", "season", "sport", "headers", "Bio", "College"]

In [83]:
harvard = harvard[harvardColumns]

In [84]:
harvard.shape

(4270, 19)

In [85]:
yale.shape

(3004, 19)

In [86]:
# Normalize column names
normColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", 
               "Class", "Active Seasons", "Sport", "Misc", "Bio", "College"]
yale.columns = normColumns
harvard.columns = normColumns

In [87]:
yale.head()

,Name,B/T,City,Events,High School,Hometown,Ht.,No.,Position,Region,S/C,Weapon,Wt.,Class,Active Seasons,Sport,Misc,Bio,College
0,A.J. Haase,NaN,Bonne Terre,NaN,North County,"Bonne Terre, MO / North County",6-4,NaN,TE,MO,None,NaN,245.0,Fr.,2006-10,Football,None,2010-11: NCAA East Regional All-TournamentTeam...,Yale
1,AJ Edwards,NaN,Seattle,NaN,South Kent School,"Seattle, Wash. / South Kent School",6-5,25.0,G,Wash.,None,NaN,190.0,Fr.,2013-17,Basketball,None,2010: Appeared in five games... Recorded a 9.0...,Yale
2,Aarica West,NaN,Buena Park,NaN,Vista Murrieta,"Buena Park, Calif. / Vista Murrieta",5-10,31.0,G/F,Calif.,None,NaN,NaN,Freshman,2009-12,Basketball,None,2014-15 (Senior): Named captain of Yale Heavyw...,Yale
3,Aaron Ault,NaN,Altamonte Springs,NaN,Lake Brantley,"Altamonte Springs, FL / Lake Brantley",6-3,93.0,LB,FL,None,NaN,194.0,Fr.,2012-16,Football,None,Notes: Owns school record for career goals aga...,Yale
4,Aaron Bosgang,NaN,Port Washington,NaN,Paul D. Schreiber High School,"Port Washington, N.Y. / Paul D. Schreiber High...",NaN,NaN,NaN,N.Y.,None,NaN,NaN,Fr.,2016-17,Crew (Lightweight),None,2015: Played in all 10 games…Had 11 solo tackl...,Yale


In [88]:
stacked = pd.concat([yale, harvard], axis=0)

In [89]:
# Drop some of the columns we don't need
finalColumns = ["Name", "City", "High School", "Hometown",
                "Ht.", "No.", "Position", "Region", "Wt.",
                "Active Seasons", "Misc", "Bio", "College"]
stacked = stacked[finalColumns]

In [90]:
stacked.shape

(7274, 13)

In [91]:
indices = range(0, 7274)
stacked["Student_ID"] = indices

In [92]:
stacked = stacked.set_index(["Student_ID"])

In [93]:
stacked.shape

(7274, 13)

In [94]:
# Seems about right.
processedPath = os.path.join(PROJ_ROOT, "data", "processed", "player_bios_processed.csv")
stacked.to_csv(processedPath, encoding='utf-8')